In [1]:
#************************* MAIN SCRIPT **********************************
import sys, os, math, h5py, csv
import matplotlib.pyplot as plt
from scipy import integrate
import numpy as np

sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [2]:
#path of the simulation directory
#path = '/Users/ajc/FUSE/simulations/tests/ats-demos/13_reactive_transport/01_surface'
path = '/Users/ajc/FUSE4/simulations/tests/ats-demos/13_reactive_transport/04_hyporheic'
path = '/Users/ajc/FUSE/simulations/tests/ats-demos/13_reactive_transport/04_hyporheic_watershed/'

#path='/Users/ajc/FUSE4/simulations/tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/coweeta/'


In [ ]:
F= ['srun1B','srun2B','srun3B','srun4B']
#F= ['srun5B','srun6B','srun7B','srun8B']

F = ['drun-base','drun1000','drun500','drun100','drun50']
F = [c + 'C' for c in F]
Files = [os.path.join(path,f,'visdump_surface_data.h5') for f in F]
vars = ['surface-total_component_concentration', 'surface-total_component_concentration']
Suffix = ['.cell.Tracer1', '.cell.Tracer2']


In [ ]:
#Colors = ['r','k','b','c','m']
Colors = ['k','c','b','g','r','grey','y']
F_L = ['Tracer 1', 'Tracer 2']
Taus = ['10','20', '30','40']
Labels = ['stream length', '1000m','500m', '100m', '50m']
#Labels = Labels[::-1]

In [ ]:
%matplotlib qt
outfile="/Users/ajc/Core/PostProcessing/ReactiveTransport/Hyporheic/Coweeta/June4/"

def plot_surface_outlet(cell_id=-1,fig_name='fig'):
    fig, axs = plt.subplots(1,2, figsize=(10,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    
    axs = axs.ravel()
    
    for i, v in enumerate(vars):
        #plt.subplot(111)
        for j,f in enumerate(Files):
            print(j,f)
            data = np.array(readdata.GetSurfVarFromVis(f,v,cellid=cell_id,suffix=Suffix[i])[v])
            
            Time = np.array([120*i/3600. for i in range(len(data))])
            #axs[i].plot(Time, 1.e-3*data,color=Colors[j],label=F_L[i]) # convert ATS moles/second to Pflotran moles/litre
            axs[i].plot(Time, 1.e-3*data,color=Colors[j],label=Labels[j])#Taus[j]) # convert ATS moles/second to Pflotran moles/litre
        axs[i].set_ylabel(vars[i],fontsize=12)

        axs[i].set_xlabel('Time [h]',fontsize=12)

        axs[i].legend(loc='best',fontsize=10)
        print (F_L[i])
        axs[i].set_title("Radioactive decay : %s"%F_L[i],fontsize=12)
        axs[i].ticklabel_format(style='sci', axis='y', scilimits=(0,0),useMathText=True)
        
        #axs[i].set_ylim(0,2e-2)

    plt.tight_layout(h_pad=1)
    
    plt.savefig(outfile + fig_name  + "-outlet.png",dpi=100)
    plt.show()

In [ ]:
plot_surface_outlet(cell_id=0, fig_name='fig3')

In [ ]:

# alpha = 1.7e-5 : inputfile = decay_ingrowth_coweeta_1.xml
#srun1A: stream rate = 0 , subgrid with rate = 1 * 1.155e-4 , mean = 0.5, taus = 10
#srun2A: stream rate = 0 , subgrid with rate = 1 * 1.155e-4 , mean = 0.5, taus = 20
#srun3A: stream rate = 0 , subgrid with rate = 1 * 1.155e-4 , mean = 0.5, taus = 30
#srun4A: stream rate = 0 , subgrid with rate = 1 * 1.155e-4 , mean = 0.5, taus = 40

# alpha = 1.7e-5 : inputfile = decay_ingrowth_coweeta_2.xml
#srun1B: stream rate = 1.155e-4 , subgrid with rate = 1.155e-4 , mean = 0.5, taus = 10
#srun2B: stream rate = 1.155e-4 , subgrid with rate = 1.155e-4 , mean = 0.5, taus = 20
#srun3B: stream rate = 1.155e-4 , subgrid with rate = 1.155e-4 , mean = 0.5, taus = 30
#srun4B: stream rate = 1.155e-4 , subgrid with rate = 1.155e-4 , mean = 0.5, taus = 40


#*************
# alpha = 1.7e-5 : inputfile = decay_ingrowth_coweeta_3.xml
#srun5B: stream rate = 1.155e-4 , subgrid with rate = 1 * 1.155e-4 , mean = 5, taus = 10
#srun6B: stream rate = 1.155e-4 , subgrid with rate = 1 * 1.155e-4 , mean = 5, taus = 20
#srun7B: stream rate = 1.155e-4 , subgrid with rate = 1 * 1.155e-4 , mean = 5, taus = 30
#srun8B: stream rate = 1.155e-4 , subgrid with rate = 1 * 1.155e-4 , mean = 5, taus = 40


In [ ]:
def subgrid_C(sg_file,sg_var, suff):
        sgC_data = readdata.GetSurfVarFromVis_All(sg_file,sg_var,suffix=suff)
        sgConcentration = sgC_data[sg_var]
        return sgConcentration
    
def plot_surface_subgrid(ncells=-1):
    fig, axs = plt.subplots(1,3, figsize=(14,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    plt.tight_layout()
    
    sum1 = 0
    
    for i, v in enumerate(vars):
        plt.subplot(131)

        dat1 = np.array(readdata.GetSurfVarFromVis(Files[0],v,cellid=ncells,suffix=Suffix[0])[v])
        dat2 = np.array(readdata.GetSurfVarFromVis(Files[0],v,cellid=ncells,suffix=Suffix[1])[v])
        data = dat1 + dat2
        Time = np.array([900*i/3600. for i in range(len(data))])
        if i ==0:
            plt.plot(Time, 1.e-3*data,color=Colors[2],label='total concent. %s'%(np.round(1e-3*data[-1],5)))
        plt.plot(Time, 1.e-3*dat1,color=Colors[0],label='%s: %s'%(F_L[0],np.round(1e-3*dat1[-1],5)))
        plt.plot(Time, 1.e-3*dat2,color=Colors[1],label='%s: %s'%(F_L[1],np.round(1e-3*dat2[-1],5)))
        sum1 = sum1 + 1e-3*data[-1]
        print ('surface:', sum1, 1e-3*data[-1])
        plt.ylabel(vars[i])

        plt.xlabel('Time [h]')
        plt.title('stream outlet, x = %s'%ncells)
        plt.legend()
        break
    plt.subplot(132)
    xnum = 100
    R = [0, ncells]

    n_taus = 20
    for i,x1 in enumerate(R):

        sg_infile = os.path.join(path,F[0],'visdump_subgrid_%s_data.h5'%x1)
        sg_var='subgrid_%s-total_component_concentration'%x1

        dat1 = subgrid_C(sg_file=sg_infile,sg_var=sg_var, suff=Suffix[0])
        dat2 = subgrid_C(sg_file=sg_infile,sg_var=sg_var, suff=Suffix[1])

        if (i ==1):
            dat11a = 1e-3*np.reshape(dat1,(-1,n_taus))
            dat11b = [d[0] for d in dat11a] #first cell in Tau space
            dat11c = [d[-1] for d in dat11a] # last cell in Tau space
            
            plt.plot(Time, dat11b,'-',color=Colors_sg[i-1],label='sg_%s[0]_Tc1 =%s)'%(R[i],np.round(dat11b[-1],4)))
            plt.plot(Time, dat11c,'-',color=Colors_sg[i],label='sg_%s[19]_Tc1 = %s)'%(R[i],np.round(dat11c[0],4)))


            dat21a = 1e-3*np.reshape(dat2,(-1,n_taus))
            dat21b = [d[0] for d in dat21a]
            dat21c = [d[-1] for d in dat21a]
            
            plt.plot(Time, dat21b,'-',color=Colors_sg[i+1],label='sg_%s[0]_Tc2 = %s)'%(R[i],np.round(dat21b[-1],4)))
            plt.plot(Time, dat21c,'-',color=Colors_sg[i+2],label='sg_%s[19]_Tc2 = %s)'%(R[i],np.round(dat21c[0],4)))

    plt.title('subgrid. Tracer 1 and 2 at sg_cell 0')
    plt.legend(loc='best', fontsize=9)
    plt.subplot(133)
    
    for i,x1 in enumerate(R):

        sg_infile = os.path.join(path,F[0],'visdump_subgrid_%s_data.h5'%x1)
        sg_var='subgrid_%s-total_component_concentration'%x1
        dat1 = subgrid_C(sg_file=sg_infile,sg_var=sg_var, suff=Suffix[0])#[-1][::-1]
        dat2 = subgrid_C(sg_file=sg_infile,sg_var=sg_var, suff=Suffix[1])#[-1][::-1]

        if (i ==1 or i ==0):
            dat11a = 1e-3*np.reshape(dat1,(-1,n_taus))
            dat11 = dat11a[-1]
            
            dat21a = 1e-3*np.reshape(dat2,(-1,n_taus))
            dat21 = dat21a[-1]
            
            plt.plot(dat11,'*',color=Colors_sg[i],label='sg_%s  (Tc1. %s)'%(x1,np.round(np.max(dat11),4)))
            plt.plot(dat21,'.',color=Colors_sg[i],label='sg_%s  (Tc2. %s)'%(x1,np.round(np.max(dat21),4)))
            plt.xlabel('# cells in Tau space')
        
    plt.title('subgrid')


    plt.legend(loc='best', fontsize=9)
    plt.show()


In [ ]:
plot_surface_subgrid(ncells=9)

In [ ]:
there is no reason to believe it will not work
I will echo your comment
We are working with one data set, but are not marry to it

we could not do it without your help

there will be bumps along the road

research has shown groundwater is an important link to the lower atmosphere.  
Both projects seem like a great opportunity to study this— are there plans for this?